# Read and Process Data

Our data are the 2022 PISA student responses from Canada and the United States. The data from PISA are for all countries and are in .sav form. The first steps in our analysis are reading and processing the data in the following stages:

1. Read data in
2. Filter to just US/Canada responses
3. Filter to non-NA targets
4. Calculate missingness proportions for our items of interest
5. Perform imputation on missing values

I performed actions 1 through 3 in RStudio locally, because the jupyter kernel could not handle the 1.9GB original file. The filtered USA/CAN file is around 3.1MB. 

# Load Dependencies

Note you may have to install some packages; check session info to see what we're running.

In [1]:
# you might need to run these to install packages
!pip install pyreadstat
!pip install session_info
!pip install polars

In [2]:
# import dependencies
import session_info
import numpy as np
import pandas as pd
import polars as pl

In [3]:
# show session info
session_info.show()

# Load and Inspect Data

Recall that the data has already been filtered to only USA and CAN country codes.

In [4]:
dat = pl.read_csv("pisa2022_usacan.csv", null_values = ["NA", "null"])

In [5]:
# Explore data
print(dat.head())
print(f'Data shape: {dat.shape}')
print(dat.describe())
print(dat.columns)

shape: (5, 76)
┌─────┬────────────┬───────┬───────────┬───┬────────────┬────────────┬────────────┬────────────┐
│     ┆ ST352Q06JA ┆ AGE   ┆ ST004D01T ┆ … ┆ ST348Q05JA ┆ ST348Q06JA ┆ ST348Q07JA ┆ ST348Q08JA │
│ --- ┆ ---        ┆ ---   ┆ ---       ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│ i64 ┆ i64        ┆ f64   ┆ i64       ┆   ┆ i64        ┆ i64        ┆ i64        ┆ i64        │
╞═════╪════════════╪═══════╪═══════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 1   ┆ 2          ┆ 15.58 ┆ 1         ┆ … ┆ 4          ┆ 3          ┆ null       ┆ null       │
│ 2   ┆ 2          ┆ 16.17 ┆ 2         ┆ … ┆ null       ┆ 4          ┆ 4          ┆ 4          │
│ 3   ┆ 4          ┆ 15.58 ┆ 2         ┆ … ┆ 3          ┆ 2          ┆ 2          ┆ 2          │
│ 4   ┆ 4          ┆ 15.42 ┆ 2         ┆ … ┆ null       ┆ 1          ┆ null       ┆ 1          │
│ 5   ┆ 2          ┆ 15.75 ┆ 2         ┆ … ┆ null       ┆ null       ┆ null       ┆ 2          │
└─────┴────────

In [6]:
# number of NA values for outcome
print(f'Non-null values for outcome: {len(dat["ST352Q06JA"])}')

Non-null values for outcome: 11181


# Handling Feature Missingness

In the R script, we filtered our target to non-null values. We did not handle feature missingness, however. First, we report the missingness for each of our features, then conduct mean imputation using the `polars` function `fill_null()`.

In [7]:
# First, let's look at the data types
print(dat.describe)

# Extract the rows that are totally null
nulls = dat.null_count()
nulls

<bound method DataFrame.describe of shape: (11_181, 76)
┌───────┬────────────┬───────┬───────────┬───┬────────────┬────────────┬────────────┬────────────┐
│       ┆ ST352Q06JA ┆ AGE   ┆ ST004D01T ┆ … ┆ ST348Q05JA ┆ ST348Q06JA ┆ ST348Q07JA ┆ ST348Q08JA │
│ ---   ┆ ---        ┆ ---   ┆ ---       ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│ i64   ┆ i64        ┆ f64   ┆ i64       ┆   ┆ i64        ┆ i64        ┆ i64        ┆ i64        │
╞═══════╪════════════╪═══════╪═══════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 1     ┆ 2          ┆ 15.58 ┆ 1         ┆ … ┆ 4          ┆ 3          ┆ null       ┆ null       │
│ 2     ┆ 2          ┆ 16.17 ┆ 2         ┆ … ┆ null       ┆ 4          ┆ 4          ┆ 4          │
│ 3     ┆ 4          ┆ 15.58 ┆ 2         ┆ … ┆ 3          ┆ 2          ┆ 2          ┆ 2          │
│ 4     ┆ 4          ┆ 15.42 ┆ 2         ┆ … ┆ null       ┆ 1          ┆ null       ┆ 1          │
│ 5     ┆ 2          ┆ 15.75 ┆ 2         ┆ … ┆ null  

,ST352Q06JA,AGE,ST004D01T,DURECEC,REPEAT,MISSSC,SKIPPING,TARDYSD,EXPECEDU,SISCO,MATHMOT,MATHEASE,MATHPREF,EXERPRAC,STUDYHMW,WORKPAY,WORKHOME,INFOSEEK,BULLIED,FEELSAFE,TEACHSUP,RELATST,SCHRISK,BELONG,GROSAGR,ANXMAT,MATHEFF,MATHEF21,MATHPERS,FAMCON,ASSERAGR,COOPAGR,CURIOAGR,EMOCOAGR,EMPATAGR,PERSEVAGR,…,CREATFAM,CREATSCH,CREATEFF,CREATOP,IMAGINE,OPENART,CREATAS,CREATOOS,ESCS,ST352Q01JA,ST352Q02JA,ST352Q03JA,ST352Q04JA,ST352Q05JA,ST352Q07JA,ST352Q08JA,ST354Q02JA,ST354Q03JA,ST354Q07JA,ST354Q08JA,ST354Q09JA,ST353Q01JA,ST353Q02JA,ST353Q03JA,ST353Q04JA,ST353Q05JA,ST353Q06JA,ST353Q07JA,ST353Q08JA,ST348Q01JA,ST348Q02JA,ST348Q03JA,ST348Q04JA,ST348Q05JA,ST348Q06JA,ST348Q07JA,ST348Q08JA
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,22,2964,69,78,97,140,540,625,97,107,93,43,42,80,59,182,66,44,9140,9126,9126,86,11181,162,163,240,180,183,2289,2227,2287,2263,2255,2259,…,11181,11181,11181,11181,11181,11181,11181,11181,34,4798,4861,4788,4812,4839,4799,4887,5717,5723,5704,5773,5744,4316,4345,4264,4265,4273,4463,4406,4273,4373,4340,4328,4283,4325,4354,4347,4315


In [8]:
# calculate the proportion of missingness
proportion_missing = {col: [nulls[col][0] / len(dat)] for col in dat.columns}
proportion_missing

{'': [0.0],
 'ST352Q06JA': [0.0],
 'AGE': [0.0],
 'ST004D01T': [0.0019676236472587426],
 'DURECEC': [0.2650925677488597],
 'REPEAT': [0.006171183257311511],
 'MISSSC': [0.00697612020391736],
 'SKIPPING': [0.008675431535640819],
 'TARDYSD': [0.012521241391646544],
 'EXPECEDU': [0.04829621679635095],
 'SISCO': [0.05589839906985064],
 'MATHMOT': [0.008675431535640819],
 'MATHEASE': [0.00956980592075843],
 'MATHPREF': [0.008317681781593774],
 'EXERPRAC': [0.003845809856005724],
 'STUDYHMW': [0.003756372417493963],
 'WORKPAY': [0.0071549950809408815],
 'WORKHOME': [0.0052768088721939005],
 'INFOSEEK': [0.016277613809140508],
 'BULLIED': [0.0059028709417762275],
 'FEELSAFE': [0.003935247294517485],
 'TEACHSUP': [0.8174581879974957],
 'RELATST': [0.816206063858331],
 'SCHRISK': [0.816206063858331],
 'BELONG': [0.007691619712011448],
 'GROSAGR': [1.0],
 'ANXMAT': [0.014488865038905285],
 'MATHEFF': [0.014578302477417046],
 'MATHEF21': [0.021464985242822646],
 'MATHPERS': [0.016098738932116986]

In [9]:
# create list of totally missing variables
totally_missing = []

# extract the keys (features) with totally missing data
for k in proportion_missing.keys():
    if proportion_missing[k][0] == 1.0:
        totally_missing.append(k)

# totally_missing = [key for key in proportion_missing if proportion_missing[key] == 1.0]
print(totally_missing)

['GROSAGR', 'CREATFAM', 'CREATSCH', 'CREATEFF', 'CREATOP', 'IMAGINE', 'OPENART', 'CREATAS', 'CREATOOS']


In [10]:
# filter out totally missing variables
dat_nomiss = dat.drop(totally_missing)

# examine data types
dat_nomiss.schema

OrderedDict([('', Int64),
             ('ST352Q06JA', Int64),
             ('AGE', Float64),
             ('ST004D01T', Int64),
             ('DURECEC', Int64),
             ('REPEAT', Int64),
             ('MISSSC', Int64),
             ('SKIPPING', Int64),
             ('TARDYSD', Int64),
             ('EXPECEDU', Int64),
             ('SISCO', Int64),
             ('MATHMOT', Int64),
             ('MATHEASE', Int64),
             ('MATHPREF', Int64),
             ('EXERPRAC', Int64),
             ('STUDYHMW', Int64),
             ('WORKPAY', Int64),
             ('WORKHOME', Int64),
             ('INFOSEEK', Float64),
             ('BULLIED', Float64),
             ('FEELSAFE', Float64),
             ('TEACHSUP', String),
             ('RELATST', String),
             ('SCHRISK', String),
             ('BELONG', Float64),
             ('ANXMAT', Float64),
             ('MATHEFF', Float64),
             ('MATHEF21', Float64),
             ('MATHPERS', Float64),
             ('FAMCON'

`TEACHSUP`, `RELATST`, and `SCHRISK` are all string variables. Let's examine those columns.

In [11]:
dat_nomiss[["TEACHSUP", "RELATST", "SCHRISK"]]

TEACHSUP,RELATST,SCHRISK
str,str,str
null,null,null
null,null,null
null,null,null
null,null,null
null,null,null
…,…,…
"""0.8211""","""0.4368""","""-0.6386"""
"""-0.5635""","""-0.2755""","""1.2046"""
"""0.4357""","""-0.257""","""1.6441"""


Okay, so they're accidentally strings, let's convert them to floats.

In [12]:
df = dat_nomiss.with_columns([
    pl.col("TEACHSUP").cast(pl.Float64),
    pl.col("RELATST").cast(pl.Float64),
    pl.col("SCHRISK").cast(pl.Float64),
])
df.schema

OrderedDict([('', Int64),
             ('ST352Q06JA', Int64),
             ('AGE', Float64),
             ('ST004D01T', Int64),
             ('DURECEC', Int64),
             ('REPEAT', Int64),
             ('MISSSC', Int64),
             ('SKIPPING', Int64),
             ('TARDYSD', Int64),
             ('EXPECEDU', Int64),
             ('SISCO', Int64),
             ('MATHMOT', Int64),
             ('MATHEASE', Int64),
             ('MATHPREF', Int64),
             ('EXERPRAC', Int64),
             ('STUDYHMW', Int64),
             ('WORKPAY', Int64),
             ('WORKHOME', Int64),
             ('INFOSEEK', Float64),
             ('BULLIED', Float64),
             ('FEELSAFE', Float64),
             ('TEACHSUP', Float64),
             ('RELATST', Float64),
             ('SCHRISK', Float64),
             ('BELONG', Float64),
             ('ANXMAT', Float64),
             ('MATHEFF', Float64),
             ('MATHEF21', Float64),
             ('MATHPERS', Float64),
             ('FAMC

In [14]:
# mean impute and write to csv
df_mean_imputed = df.with_columns(pl.all().fill_null(pl.all().mean()))
df_mean_imputed.write_csv("pisa_mean.csv")

# median impute and write to csv
df_median_imputed = df.with_columns(pl.all().fill_null(pl.all().median()))
df_median_imputed.write_csv("pisa_median.csv")